In [1]:
import sys
import numpy as np
import pandas as pd
from utils import *
sys.path.append('../')
from bayes_ensemble.bayes_ensemble import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_preds_all.head()

,date,pred,lower,upper,state,model_id
0,2022-10-09,110.690113,64.291013,219.826948,AC,21
1,2022-10-16,144.952846,77.564037,279.722708,AC,21
2,2022-10-23,163.613937,92.915072,276.591399,AC,21
3,2022-10-30,170.240443,87.197631,378.407811,AC,21
4,2022-11-06,215.117888,111.835667,445.024983,AC,21


In [3]:
excluded_models

[(21, 'MG'),
 (21, 'ES'),
 (25, 'MG'),
 (21, 'RR'),
 (21, 'RO'),
 (21, 'AL'),
 (25, 'CE')]

In [4]:
models_by_state.head()

,state,model_id
0,AC,"[34, 21, 22, 27, 28, 30]"
1,AL,"[34, 22, 27, 28, 30]"
2,AM,"[34, 21, 22, 27, 28, 30]"
3,AP,"[34, 21, 22, 27, 28, 30]"
4,BA,"[34, 21, 22, 27, 28, 30]"


#### Apply the ensemble with linear mix crps

In [5]:
%%time 
df_for_val_base = pd.DataFrame()

df_23_24 = pd.DataFrame()

df_25 = pd.DataFrame()

df_w = pd.DataFrame()

metric = 'log_score'

for state in states_BR: 
    
    data_23, data_24, preds_23, preds_24, preds_25, models = format_data(state, models_by_state, data_all, df_preds_all)
    
    ens_23 = Ensemble_linear(df= preds_23,
            order_models = models)

    weights_23 = ens_23.compute_weights(df_obs=data_23, metric = metric)
    
    df_23_in = ens_23.apply_ensemble(weights = weights_23['weights'])

    ens_24 = Ensemble_linear(df= preds_24,
            order_models = models)

    df_24_out = ens_24.apply_ensemble(weights = weights_23['weights'])

    df_23_24_ = pd.concat([df_23_in, df_24_out], ignore_index = True)

    df_23_24_['state'] = state
    
    df_23_24 = pd.concat([df_23_24, df_23_24_], ignore_index = True)

    #### Forecast 2025
    
    weights_24 = ens_24.compute_weights(df_obs=data_24, metric = metric)

    ens_25 =  Ensemble_linear(df = preds_25,
            order_models = models)

    df_25_23 = ens_25.apply_ensemble(weights = weights_23['weights'])

    df_25_24 = ens_25.apply_ensemble(weights = weights_24['weights'])


    df_25_ = df_25_23.rename(columns = {'pred':'pred_ensemble_23',
                           'lower':'lower_ensemble_23', 
                           'upper':'upper_ensemble_23'}).merge(df_25_24.rename(columns = {'pred':'pred_ensemble_24',
                           'lower':'lower_ensemble_24', 
                           'upper':'upper_ensemble_24'}), left_on = 'date', right_on = 'date' )

    df_25_['state'] = state

    df_25 = pd.concat([df_25, df_25_], ignore_index = True)

    df_w_ = pd.DataFrame([[weights_23['weights'], 2023, state], 
             [weights_24['weights'], 2024, state]], columns = ['weights', 'year', 'state'])


    df_w = pd.concat([df_w, df_w_])
    

CPU times: user 17min 21s, sys: 25.1 s, total: 17min 46s
Wall time: 17min 39s


In [7]:
df_25.head()

,pred_ensemble_23,lower_ensemble_23,upper_ensemble_23,date,pred_ensemble_24,lower_ensemble_24,upper_ensemble_24,state,model_id
0,202.032747,44.998537,907.044750,2024-10-06,152.642480,53.141678,741.399309,AL,lin_ln
1,199.109809,44.001052,900.958264,2024-10-13,155.789569,50.514998,735.873350,AL,lin_ln
2,199.655978,43.995941,906.009419,2024-10-20,154.153851,52.826272,739.425339,AL,lin_ln
3,201.802529,44.997919,904.988094,2024-10-27,148.138670,51.166532,740.040435,AL,lin_ln
4,199.318911,43.998081,902.909620,2024-11-03,165.672816,50.434339,737.354357,AL,lin_ln


In [9]:
df_23_24.to_csv(f'../predictions/ensemble_2023_2024_{metric}_lin_ln.csv', index = False)
df_25.to_csv(f'../predictions/ensemble_2025_{metric}_lin_ln.csv', index = False)
df_w.to_csv(f'../predictions/weights_{metric}_lin_ln.csv',index = False)